In [ ]:
import json
import PyPDF2
from os import listdir

#Ham doi ten tu cac FileFail thanh File cu                    
def getFileFail(name, listfile):
    for i in listfile:
        if i[12:16] == name[4:8]: return i
    return False

#DS 616 file pdf ban dau
list_files = listdir(r'C:\Users\an\Desktop\New folder')
#DS cac file pdf khong co du lieu ban dau
list_filesN = listdir(r'C:\Users\an\Desktop\NewFile')


# File chua 420 file pdf lay lan 1
File = open('successFile1.txt').read().split('\n')[:-1]

# File chua 126 file pdf lay lan 2
FileFail = open('successFileFail.txt').read().split('\n')[:-1]
# File chua 126 file pdf lay lan 2 da dc doi ten
newFile = []
for i in FileFail:
    newFile.append(getFileFail(i, list_files))
    


In [ ]:
#Ham lay text de kiem tra
def Text(path, num):
    a = r'C:\Users\an\Desktop\New folder\{}'.format(path)
    file = open(a, 'rb')
    fileReader = PyPDF2.PdfFileReader(file)
    page = fileReader.getPage(num)
    text = page.extractText().lower()
    newtext = ' '.join(text.strip().split('\n'))
    newtext1 = str(newtext.encode("ascii", "ignore"))
    return newtext1.strip()
#Ham lay text(str) va otext(list) de tach du lieu va ghep lai cau hoan chinh
def getText(path, num):
    bien = getTittle(path)
    text = Text(path, num)[bien[0]:bien[1]]
    oText = text.strip().split()
    newtext1= ''.join(oText)
    return oText, newtext1

### loai bo noi dung bi trung o dau va cuoi moi trang.
def getTittle(path):
    text1 = Text(path, 2)
    text2 = Text(path, 3)
    if (text1[0] == text2[0]) and (text1[-5:] == text2[-5:]):
        for i in range(300):
            if(text1[i] != text2[i]):
                bien1 = i
                break
        count = 1
        for i in range(-1,-300,-1):
            if(text1[i] != text2[i]):
                if(count == 0):
                    bien2 = i
                    return bien1, bien2+1
                else:
                    count-=1
    else:
        count = 1
        for i in range(250):
            if(text1[i] != text2[i]):
                if(count == 0):
                    return i, None
                else:
                    count-=1
                    
# tra ve mang chua text khong co khoang trang dung de dua ve text ban dau                    
def timMang(mang, text):
    for i in range(len(mang)):
        if len(''.join(mang[i:])) <= len(text):
            return mang[i:]
def timMangSau(mang, text):    
    for i in range(len(mang)):
        if len(''.join(mang[:i])) == len(text):
            return mang[:i]
        if len(''.join(mang[:i])) >= len(text):
            return mang[:i-1]

def checknum(text):
    try:
        return int(text[-4:-2])
    except:
        try:
            return int(text[-3:-1])
        except:
            try:
                return int(text[-2:])
            except:
                try:
                    return int(text[-1:])
                except:
                    return False


In [ ]:
f = open('textTOC.txt')
textTOC = json.load(f)

newdic1 = dict()
nonList = []
nonConclu = []
for  i in File+newFile:
    if 'listofstudiestobegenerated' in textTOC[i]:
        bien = textTOC[i].split('listofstudiestobegenerated',1)
        if len(bien) == 2:
            newdic1[i] = bien[1]
        if 'conclusionsandrecommendations' not in textTOC[i]: nonConclu.append(i)
    elif 'conclusionsandrecommendations' in textTOC[i]:
        nonList.append(i)
        bien = textTOC[i].split('conclusionsandrecommendations',1)
        if len(bien) == 2:
            newdic1[i] = bien[1]
    else:
        nonList.append(i)
        nonConclu.append(i)
        bien = textTOC[i].split('concerns',1)
        if len(bien) == 2:
            newdic1[i] = bien[1]

for i in newdic1:
    if 'references' in newdic1[i] and 'appendices' in newdic1[i]:
        bien = newdic1[i].split('appendi',1)
        if len(bien) == 2:
            newdic1[i] = bien[0]
    elif 'references' in newdic1[i] and 'abbreviations' in newdic1[i]:
        bien = newdic1[i].split('abbreviations',1)
        if len(bien) == 2:
            newdic1[i] = bien[0]
    else:
        bien = newdic1[i].split('appendi',2)
        if len(bien) == 3:
            newdic1[i] = bien[0] + bien[1]
        else:
            bien = newdic1[i].split('abbreviations',1)
            if len(bien) == 2:
                newdic1[i] = bien[0]
page1 = dict()
for i in newdic1:
    bien = newdic1[i].split('conclusionsandrecommendations', 1)
    if len(bien) == 2:
        page1[i] = checknum(bien[0])
        nonConclu.append(i)
    else:
        bien = newdic1[i].split('particularconditionsproposedtobetakenintoaccount', 1)
        if len(bien) == 2:
            page1[i] = checknum(bien[0])
            nonConclu.append(i)
        else:
            bien = newdic1[i].split('1.issuesthatcouldnotbefinalised', 1)
            if len(bien) == 2:
                check = checknum(bien[0])
                if type(check) != int:
                    bien = newdic1[i].split('9.1.concernsfortherepresentativeusesevaluated', 1)
                    check = checknum(bien[0])
                    page1[i] = check
                    nonConclu.append(i)
                else:    
                    page1[i] = checknum(bien[0])
                    nonConclu.append(i)
            else:
                bien = newdic1[i].split('3.concerns', 1)
                if len(bien) == 2:
                    check = checknum(bien[0])
                    if type(check) != int: print(newdic1[i])
                    page1[i] = checknum(bien[0])
                    nonConclu.append(i)
                else:
                    bien = newdic1[i].split('issuesthatcouldnotbefinalised', 1)
                    if len(bien) == 2:
                        check = checknum(bien[0])
                        if type(check) != int: print(newdic1[i])
                        page1[i] = check
                        nonConclu.append(i)
                    else:                           
##################################################################################################################################
                        bien = newdic1[i].split('1.issuesthatcouldnotbenalised', 1)
                        if len(bien) == 2:
                            check = checknum(bien[0])
                            if type(check) != int: 
                                bien = newdic1[i].split('9.1.concernsfortherepresentativeusesevaluated', 1)
                                check = checknum(bien[0])
                                page1[i] = check
                                nonConclu.append(i)
                            else:
                                page1[i] = check
                                nonConclu.append(i)
                        else:
                            bien = newdic1[i].split('9.1issuesthatcouldnotbenalised', 1)
                            if len(bien) == 2:
                                check = checknum(bien[0])
                                if type(check) != int: print(bien[0])
                                page1[i] = check
                                nonConclu.append(i)
                            else:
                                bien = newdic1[i].split('10.1.concernsfortherepresentativeusesevaluated', 1)
                                if len(bien) == 2:
                                    check = checknum(bien[0])
                                    if type(check) != int: print(bien[0])
                                    page1[i] = check
                                    nonConclu.append(i)
                                else:    
                                    print(i)
                                    print(newdic1[i])
                                    print('\n')
                                    
page2 = dict()
for i in newdic1:
    check = checknum(newdic1[i])
    if type(check) != int:
        text = newdic1[i].split('9.3.')[0]
        check = checknum(text)
        if type(check) == int:
            page2[i] = check        
        else:
            check = checknum(newdic1[i].split('appendixb')[0])
            page2[i] = check 
    else:
        page2[i] = check
page1['j.efsa.2020.6006.pdf'] = 10    

newTT = dict()
onewTT = dict()

for i in newdic1:
    text = ''
    otext = []
    for j in range(page1[i]-1, page2[i]-1):
        bien = getText(i, j)
        text+= bien[1]
        otext+= bien[0]
    newTT[i] = text
    onewTT[i] = otext
    
with open('newTT.txt', 'w') as convert_file:
     convert_file.write(json.dumps(bien))
with open('onewTT.txt', 'w') as convert_file:
     convert_file.write(json.dumps(obien))

listList = []
listConclu = []
listConcern = []

f = open('newTT.txt')
bien = json.load(f)
f1 = open('onewTT.txt')
obien = json.load(f1)

for i in File + newFile:
    if i in nonConclu:
        if i in nonList:
            listList.append(None)
            listConclu.append(None)
            listConcern.append(' '.join(obien[i]))
        else:
            array = bien[i].split('criticalareasofconcern', 1)
            if len(array) == 2:
                listConclu.append(None)
                listList.append(' '.join(timMangSau(obien[i], array[0])))
                listConcern.append(' '.join(timMang(obien[i], array[1])))
            else:
                array = bien[i].split('concerns', 1)
                if len(array) == 2:
                    listConclu.append(None)
                    listList.append(' '.join(timMangSau(obien[i], array[0])))
                    listConcern.append(' '.join(timMang(obien[i], array[1])))
                else:
                    if i == 'j.efsa.2010.1526.pdf':
                        array = bien[i].split("uses.'the")
                        listConclu.append(None)
                        listList.append(' '.join(timMangSau(obien[i], array[0])) + 'uses.')
                        listConcern.append("the" +' '.join(timMang(obien[i], array[1])))
                    if i == 'j.efsa.2010.1708.pdf':
                        array = bien[i].split("operations.")
                        listConclu.append(None)
                        listList.append(' '.join(timMangSau(obien[i], array[0])) + 'operations.')
                        listConcern.append(' '.join(timMang(obien[i], array[1])))
    else:
        if i in nonList:
            print(i)
        else:
            array = bien[i].split('criticalareasofconcern', 1)
            if len(array) == 2:
                listConcern.append(' '.join(timMang(obien[i], array[1])))                
                text = array[0]
                otext = timMangSau(obien[i], array[0])
                array = text.split('conclusionsandrecommendations')
                if len(array) == 2:
                    listConclu.append(' '.join(timMang(otext, array[1])))
                    listList.append(' '.join(timMangSau(otext, array[0])))
                else:
                    array = text.split('onclusionsandrecommendations')
                    if len(array) == 2:
                        listConclu.append(' '.join(timMang(otext, array[1])))
                        listList.append(' '.join(timMangSau(otext, array[0])))
                    else:
                        array = text.split('onthebasis')
                        listConclu.append("the conclusion was reached on the basis" + ' '.join(timMang(otext, array[1])))
                        listList.append(' '.join(timMangSau(otext, array[0])))
            else:
                array = bien[i].split('riticalareasofconcern', 1)
                if len(array) == 2:
                    if i == 'j.efsa.2005.102r.pdf':
                        listConcern.append(' '.join(timMang(obien[i], array[1])).split('8. 8')[0])
                    else:
                        listConcern.append(' '.join(timMang(obien[i], array[1])))
                    text = array[0]
                    otext = timMangSau(obien[i], array[0])
                    array = text.split('conclusionsandrecommendations')
                    if len(array) == 2:
                        listConclu.append(' '.join(timMang(otext, array[1])))
                        listList.append(' '.join(timMangSau(otext, array[0])))
                    else:
                        array = text.split('onclusionsandrecommendations')
                        if len(array) == 2:
                            listConclu.append(' '.join(timMang(otext, array[1])))
                            listList.append(' '.join(timMangSau(otext, array[0])))
                else:
                    listConcern.append(None)
                    array = bien[i].split('conclusionsandrecommendations')
                    if len(array) == 2:
                        if i == 'j.efsa.2007.103r.pdf':
                            listConclu.append(' '.join(timMang(obien[i], array[1])).split('critical areas of conc')[0])
                            listList.append(' '.join(timMangSau(obien[i], array[0])))
                        else:
                            listConclu.append(' '.join(timMang(obien[i], array[1])))
                            listList.append(' '.join(timMangSau(obien[i], array[0])))
import pandas as pd
xong = {
    'ten': File+newFile,
    'tt1': listList,
    'tt2': listConclu,
    'tt3': listConcern
}

df = pd.DataFrame(list(zip(File+newFile, listList, listConclu, listConcern))
                           , columns=['Name', 'List of studies to be generated', 'Conclusions and Recommendations', 'Concerns'])
df.to_csv('data.csv', index=False)